In [93]:
import requests
import numpy as np
import json
import pandas as pd
!pip install tqdm
from tqdm import tqdm
import time
from datetime import datetime

You should consider upgrading via the 'c:\users\fedem\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [99]:
quote = "USDT" #pairs will be expressed in the value of this token, case sensitive!
destinationAllPairs = f"AllTargets_{quote}.csv" # here we will save all the pairs BASE/QUOTE
timeframe ="1d" #1m 3m 5m 15m 30m 1h 2h 4h 6h 8h 12h 1d 3d 1w 1M
destinationHistoryRaw = f"History_Assets_{quote}_{timeframe}_RAW.csv"
destinationHistoryClean = f"History_Assets_{quote}_{timeframe}_Close_CLEAN.csv"
sleepingSeconds = 0.3 #amount of time from one request to the other (to prevent rate-limit)

In [53]:
#Getting all the pairs BASE/QUOTE
url = "https://api.binance.com/api/v1/exchangeInfo"
payload={}
headers = {}
response = requests.request("GET", url, headers=headers, data=payload) #we ask binance general info 
symbols = json.loads(response.text)["symbols"] #among general info we fetch the symbols 
#among symbols we take those that trade with our quote asset and are tradable
pairs = list(filter(lambda x: x["status"]=="TRADING" and x["quoteAsset"]==quote , symbols))
dataset = pd.DataFrame(pairs)
dataset.to_csv(destinationAllPairs)

In [80]:
#we get access to the previously saved all pairs
df = pd.read_csv (destinationAllPairs)
df.symbol
datasetHistory = pd.DataFrame()

for i in tqdm(range(len(df.symbol))):
    pair = df.symbol[i]
    url= f"https://api.binance.com/api/v3/klines?symbol={pair}&interval={timeframe}&limit=1000"
    payload={}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload) #we ask binance general info 
    #Recall that the response structure is the following (EXAMPLE)
    #   [
    #     1499040000000,      // Open time
    #     "0.01634790",       // Open
    #     "0.80000000",       // High
    #     "0.01575800",       // Low
    #     "0.01577100",       // Close
    #     "148976.11427815",  // Volume
    #     1499644799999,      // Close time
    #     "2434.19055334",    // Quote asset volume
    #     308,                // Number of trades
    #     "1756.87402397",    // Taker buy base asset volume
    #     "28.46694368",      // Taker buy quote asset volume
    #     "17928899.62484339" // Ignore.
    #   ]
    #Each row of this dataset will correspond to this information at given time t, for all the considered pairs 
    datasetHistory[pair]=pd.Series(list(json.loads(response.text)[::-1]))
    #rows that are missing will be replaced by NaN
    #Most recent data will be in the first row, so that we have time-homogeneity 
    #Consider that the first row corresponds to the current market value! it might change if you rebuild the dataset within the same period
    time.sleep(sleepingSeconds)
    
datasetHistory.to_csv(destinationHistoryRaw)

100%|██████████| 339/339 [04:07<00:00,  1.37it/s]


In [100]:
#we clean the dataset indexing rows through timestamps and taking just the close values of the candlestick values 
#We get all the seventh elements of the arrays in the first column (they correspond to timestamps)
#We also set timestamps in readable format
timestamps = pd.to_datetime([ x[0]*1000000 for x in datasetHistory.iloc[:,0]])
timestamps 
cleanDatasetHistory = pd.DataFrame(index=timestamps, columns=datasetHistory.columns)
#Now, for each column, we extract the fifth value of the array, which is the close value of the candle
#If the result is NaN, we return zero
for i in tqdm(range(len(cleanDatasetHistory.columns))):
    cleanDatasetHistory[cleanDatasetHistory.columns[i]]=[ x[4] if type(x)==type([]) else 0 for x in datasetHistory.iloc[:,i]]
cleanDatasetHistory.to_csv(destinationHistoryClean)

100%|██████████| 339/339 [00:00<00:00, 2337.87it/s]
